In [131]:
import pandas as pd
import numpy as np

data = pd.read_csv('grouped_data.csv')

In [132]:
data.shape

(2928, 4)

In [133]:
data['reviewText'][0]

"Perfect for filling up oil in my car which has a plastic cover over the entire engine bay and the oil input is kinda sunk down a little. Plus it can hold the quart of oil for you., Good plastic container for the price to put oil in after each change. holds 12 quarts. Afterwards you can take to a recycle center and pour it out. Only thing is I wish it had a better made lid with better seal because eventually it will wear out., It suds up good, i used this in an attached sprayer bottle on my hose and it foamed up good spraying it out. Noticed after rinsing it immediately started beading water off which is a good sign. Only time will tell on how long it will last. As far as cleaning power it does fairly good, but i've seen better. Then again it is trying to do two jobs in one. The best cleaning of a car is in steps, but for an all in one cleaner and wax i think this outperforms most., Used it to fill a tahoe and a small car, both works well and the pump doesn't even drag at all. Nicely b

In [134]:
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")


In [135]:
os.getenv("HF_TOKEN")
llm = ChatGroq(model_name = "Gemma-7b-It",groq_api_key=api_key)
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [136]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [137]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002999B638C70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000299A52054B0>, model_name='Gemma-7b-It', groq_api_key=SecretStr('**********'))

In [138]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
def store_review_text(data):
    split = text_splitter.split_text(data.lower())
    store = FAISS.from_texts(texts=split,embedding=embeddings)
    return store
def store_summary(data):
    split = text_splitter.split_text(data.lower())
    store = FAISS.from_texts(texts=split,embedding=embeddings)
    return store

In [139]:
splits = [text_splitter.split_text(x) for x in data['summary'][:1000]]
texts = [item for sublist in splits for item in sublist]


In [140]:

store = FAISS.from_texts(texts=texts,embedding=embeddings)

In [ ]:
store.similarity_search("great mask, great gloves, best gloves,")

[Document(page_content='great mask, great gloves, best gloves, wow what a jacket !!, awesome goggles'),
 Document(page_content='I love my Leather Chaps, Metal Disc Lock, Kuryakyn Electrical Power Point, Love my net., I love my Fleece Mask, Make sure you fasten your bults first before you take it for a ride., I love to ride with my Skull Face Mask.'),
 Document(page_content='The BEST!!!, FAST service, GREAT price, SUPER product !!!, GREAT GREAT GREAT, Great Goggles, Great price, Nice Gloves !!, BIG Mirror !!'),
 Document(page_content="So far so good, Great gloves for the price!, Love it! Didn't feel the cold wind this weekend at all!, Incredible !!, Works great")]

In [ ]:
store.save_local("summary_store")

In [ ]:
splits = [text_splitter.split_text(x) for x in data['reviewText'][:1000]]
texts = [item for sublist in splits for item in sublist]

review_store = FAISS.from_texts(texts=texts,embedding=embeddings)

In [ ]:
review_store.save_local('reviewText_store')